In [ ]:
!pip install semantic-kernel
!pip install psycopg
!pip install psycopg_pool

In [13]:
from typing import Tuple
import psycopg2
import psycopg

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAITextEmbedding,
    AzureChatCompletion,
    AzureTextEmbedding,
)

In [14]:
kernel = sk.Kernel()

In [15]:
completions_deployment_name = 'completions4'
embeddings_deployment_name = 'embeddings'

azure_endpoint = "YOUR_ENDPOINT"
azure_key = 'YOUR_KEY'

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(deployment_name=completions_deployment_name, endpoint=azure_endpoint, api_key=azure_key),
)

In [16]:
from semantic_kernel.connectors.memory.postgres import PostgresMemoryStore

#connect to the database
username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevilkdittaflex16.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{dbname}"

memory_store = PostgresMemoryStore(connection_string=connection_string, default_dimensionality=1536, min_pool=1, max_pool=10)

await memory_store.create_collection_async(collection_name='aboutMe')

In [17]:
kernel.add_text_embedding_generation_service(
    "text_embedding",
    AzureTextEmbedding(deployment_name=embeddings_deployment_name, endpoint=azure_endpoint, api_key=azure_key)
)

# text-embedding-ada-002 uses a 1536-dimensional embedding vector
kernel.register_memory_store(
    memory_store=memory_store
)

In [18]:
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(collection="aboutMe", id="info1", text="My name is Andrea")
    await kernel.memory.save_information_async(
        collection="aboutMe", id="info2", text="I currently work as a tour guide"
    )
    await kernel.memory.save_information_async(
        collection="aboutMe", id="info3", text="I've been living in Seattle since 2005"
    )
    await kernel.memory.save_information_async(
        collection="aboutMe",
        id="info4",
        text="I visited France and Italy five times since 2015",
    )
    await kernel.memory.save_information_async(collection="aboutMe", id="info5", text="My family is from New York")

In [19]:
await populate_memory(kernel)

# Review the memory data

- Run the following query against the Azure Database for PostgreSQL Flexible Server to see the items added to the database:

```sql
select *
from "aboutMe"
```

In [20]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search_async("aboutMe", question)

        try:
            print(f"Answer: {result[0].text}\n")
        except IndexError:
            print("No answer found\n")

In [21]:
await search_memory_examples(kernel)

Question: what's my name
Answer: My name is Andrea

Question: where do I live?
Answer: I've been living in Seattle since 2005

Question: where's my family from?
Answer: My family is from New York

Question: where have I traveled?
Answer: I visited France and Italy five times since 2015

Question: what do I do for work
Answer: I currently work as a tour guide

